Simulations to explore how well we can estimate the magnetisation of an Ising system at the level of blocks using snapshots of the system state taken at a particular stake in time.

We do the following:
1. Evaluate how well we can estimate the magnetisation at the level of blocks by sampling different fractions of nodes within blocks.
2. Evaluate the performance of different realisations of the 'snapshot influence strategy' with different sampling fractions.

Created on: 03/10/20

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import math
import random
import itertools
import matplotlib
from scipy import stats

from ising_block_level_influence import N_Block_sbm_class as NBlock
from ising_block_level_influence import projection_simplex as proj
from ising_block_level_influence import mean_field_IIM
from spatial_spin_monte_carlo import spatial_spin_monte_carlo as Spins
import Pokec_processing as  PokProc

from pokec_utils import *

In [2]:
graph = nx.read_graphml('Data/Bratislava_graph.graphml')
bratislava_profiles = pd.read_csv("Data/bratislava_profiles.csv")
coupling_graph = nx.read_graphml('Data/Bratislava_coupling.graphml')
block_data = pd.read_csv('Data/block_info.csv',converters={'Block' : ast.literal_eval})
mean_block_ages = list(block_data['average_age'])
block_sizes = list(block_data['block_size'])
block_names = list(block_data['block_name'])
block_data['age_group'] = [ 'ages_' + k.split('_')[-1] for k in list(block_data['block_name'])]
block_data.head()

,Unnamed: 0,block_name,block_size,average_age,age_group
0,0,Okolie_ages_1-17,1234,9.0,ages_1-17
1,1,Okolie_ages_18-21,1939,19.5,ages_18-21
2,2,Okolie_ages_22-28,3154,25.0,ages_22-28
3,3,Okolie_ages_29-112,3458,70.5,ages_29-112
4,4,Petrzalka_ages_1-17,1398,9.0,ages_1-17


In [3]:
gradient=1.0

def linear_field(x : np.ndarray,gradient :float) :
    return gradient*x

#Scale ages to [-1,1]:
rescaled_ages = [ (k-np.mean(mean_block_ages))/(max(mean_block_ages)-min(mean_block_ages)) for k in mean_block_ages ]

beta_c = Spins.crit_beta_sparse(graph)
bratislava_profiles_indices = bratislava_profiles.reset_index()
groups = [ bratislava_profiles_indices.loc[bratislava_profiles_indices['block']==block] for block in block_names]
groups_node_ids = [list(k['index']) for k in groups]
age_field = [linear_field(a,gradient) for a in rescaled_ages ]
age_field_map = {k:j for k,j in zip(list(block_data['age_group']),age_field)}
background_field = np.asarray([age_field_map[k] for k in list(bratislava_profiles['age_group'])])
relab_graph = nx.relabel.convert_node_labels_to_integers(graph)

Computing critical temperature


## Convergence of magnetisation

In [5]:
T=0
T_Burn = 300000
beta_f = 8.0

initial_state = -1.0*np.ones(len(graph))
examples = 10

data = pd.DataFrame()
s_frac_vals = [10**k for k in np.linspace(-4,0,10)]
print(s_frac_vals)

for sample_frac in tqdm.tqdm_notebook(s_frac_vals) : 
    for p in range(examples) : 
        block_snapshot = Run_MonteCarlo_Snapshot(relab_graph,groups_node_ids ,T, beta_f,beta_c ,T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state,frac_to_sample=sample_frac)
        
        for num,mag in enumerate(block_snapshot) :
            data=data.append(pd.DataFrame({'block':[num],'sample_frac':[sample_frac],'mag':[mag]}))
        # Also append the average magnetisation computed by summing up the blocks.
        mean_mag=np.sum([i*j for i,j in zip(block_snapshot,block_sizes)])/float(len(graph)) 
        data=data.append(pd.DataFrame({'block':['mean'],'sample_frac':[sample_frac],'mag':[mean_mag]}))

[0.0001, 0.0002782559402207126, 0.000774263682681127, 0.002154434690031882, 0.005994842503189409, 0.016681005372000592, 0.046415888336127774, 0.12915496650148828, 0.3593813663804626, 1.0]


In [6]:
data.to_csv(f"Data/snapshot_mag_eval_data_{gradient}".replace('.','-')+".csv")

## Snapshot control - sweep over sample_frac values and multiple snapshot trials

In [9]:
def mean_and_se(values) :
    return f"{np.mean(values)} +/- {stats.sem(values)} "


#Seed the random number generators:
seed = 1
random.seed(seed)
np.random.seed(seed)

Samples = 10 # For MC simulations
beta_factor=8.0
beta_f = 8.0
T=10000
T_Burn = 300000
T_snap=0
T_Burn_snap = 300000
Field_Budget=2500.0

con_names = ['no con','unif','snapshot']
eval_data = pd.DataFrame()
snapshot_controls = pd.DataFrame()

s_frac_vals = [1E-3,1E-1,1.0,1E-2,]
snapshot_samps = 15 # number of different snapshots to try.

# This doesn't need to be the outer loop as we don't need to re-compute the other evaluations
for sample_frac in tqdm.tqdm_notebook(s_frac_vals,desc='Sample fraction') :
    
    
    for q in tqdm.tqdm_notebook( range(snapshot_samps) ,desc='Snapshot')  : 
    
        initial_state = -1.0*np.ones(len(graph))
        print(f"Making snapshot for {round(sample_frac,3)}")
        block_snapshot = Run_MonteCarlo_Snapshot(relab_graph,groups_node_ids ,T_snap, beta_f,beta_c ,T_Burn=T_Burn_snap,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state,frac_to_sample=sample_frac)

        uniform_control = (Field_Budget / len(graph)) * np.ones(len(graph))

        snapshot_gradient = [ (1.0-k**2) for k in block_snapshot ] 
        snapshot_grad_map = { i:j for i,j in zip(block_names,snapshot_gradient)}
        block_names_list = list(bratislava_profiles['block'])
        snapshot_influence = np.asarray([snapshot_grad_map[k] for k in block_names_list])
        
        for index,block_name in enumerate(block_names) : 
            snapshot_controls=snapshot_controls.append(pd.DataFrame({'block':[block_name],
                                                                     'beta_factor':[beta_factor],
                                                                     'gradient':[gradient],
                                                                     'snapshot_num':[q],
                                                                     'samp_frac':[sample_frac],
                                                                     'snapshot_mag':[block_snapshot[index]],
                                                                     'snapshot_grad':[snapshot_gradient[index]]}))

        snapshot_controls.to_csv(f"Data/snapshot_controls_as_samp_grad_{gradient}".replace('.','-')+".csv")
        #Normalize:
        H_snapshot = np.sum(snapshot_influence)
        snapshot_influence = (Field_Budget/H_snapshot)*snapshot_influence

        for s in tqdm.tqdm_notebook( range(Samples) , desc='MC sim per snapshot' )  : 

            # Only run the no control and uniform on the first loop:
            # Will end up creating repeated records.
            if sample_frac == s_frac_vals[0] and q==0 :
                initial_state = -1.0*np.ones(len(graph))
                no_control = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state)

                initial_state = -1.0*np.ones(len(graph))
                unif_control_field = np.asarray([i+j for i,j in zip(uniform_control,background_field)])
                unif_control = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=unif_control_field,initial_state=initial_state)

            initial_state = -1.0*np.ones(len(graph))
            snapshot_control_field = np.asarray([i+j for i,j in zip(snapshot_influence,background_field)])
            snapshot_magnetisations = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=snapshot_control_field,initial_state=initial_state)


            means=[]
            ses=[]
            #Loop in same order as cont_names
            control_list = [no_control,unif_control,snapshot_magnetisations]
            for k in control_list :
                print(mean_and_se(k))
                means.append(np.mean(k))
                ses.append(stats.sem(k))

            eval_data = eval_data.append(pd.DataFrame({'control':con_names,'magnetisation':means,'mag_se':ses,
                                                       'beta_factor':beta_factor*np.ones(len(means)),
                                                       'gradient':gradient*np.ones(len(means)),
                                                       'samp_frac':sample_frac*np.ones(len(means)),
                                                      'snapshot_num':q*np.ones(len(means))}))
            
            
            
            eval_data.to_csv(f"Data/snapshot_as_sampfrac_data_grad_{gradient}_v2".replace('.','-')+".csv")

Making snapshot for 0.001



-0.6101971401527956 +/- 1.731473363552784e-05 
-0.586740795078088 +/- 1.628731923603448e-05 
-0.5834882360895138 +/- 1.1667648937236492e-05 



-0.6198718342235142 +/- 9.14220609914077e-06 
-0.5855595159218444 +/- 2.0439714262604796e-05 
-0.5912003583260089 +/- 1.2930776646436454e-05 



-0.611881583395308 +/- 9.384728536691929e-06 
-0.5846545128794537 +/- 2.3703222844542986e-05 
-0.5918487052937597 +/- 1.8941060577722597e-05 



-0.6114483131634102 +/- 7.2882909750735565e-06 
-0.5802876546548578 +/- 1.0001515124797284e-05 
-0.5852329659928334 +/- 1.4673604334719535e-05 


...


-0.609639666013116 +/- 9.89272810598293e-06 
-0.5725533229666688 +/- 3.078631499156743e-05 
-0.576141011425867 +/- 1.1291969684177448e-05 



-0.609639666013116 +/- 9.89272810598293e-06 
-0.5725533229666688 +/- 3.078631499156743e-05 
-0.57215495233588 +/- 1.0996584025453832e-05 



-0.609639666013116 +/- 9.89272810598293e-06 
-0.5725533229666688 +/- 3.078631499156743e-05 
-0.5762749442228382 +/- 1.5166538686774747e-05 



-0.609639666013116 +/- 9.89272810598293e-06 
-0.5725533229666688 +/- 3.078631499156743e-05 
-0.5770753025488473 +/- 1.5154123631013629e-05 



-0.609639666013116 +/- 9.89272810598293e-06 
-0.5725533229666688 +/- 3.078631499156743e-05 
-0.5801108173889529 +/- 1.4016834149425325e-05 



